# Lecture 9: Iterators and generators

- Implement iterator behavior in custom classes so they can be looped through using the same convenient syntax as built-in containers
- Use generators to create simple, custom iterators

__Reading material__: 
- [Python tutorial](https://docs.python.org/2/tutorial/) 9.9 - 9.11
- Optional: http://anandology.com/python-practice-book/iterators.html

We’re learning to write the blueprint for our own container objects, that is, objects that contain multiple elements that we can access individually and iterate over in a for loop. Python lists, tuples, sets, and dictionaries are all built-in container objects. 

In [1]:
for element in [1, 2, 3]:
    print element
for element in (1, 2, 3):
    print element
for key in {'one':1, 'two':2}:
    print key
for char in "123":
    print char

1
2
3
1
2
3
two
one
1
2
3


## Iterators
Now we’re trying to create our own.
Our custom container might have a built-in container object as an instance variable. For instance, the following __Reverse__ class has an instance variable data that is already a container. In this case, we just need to write some code that tells a for loop how to iterate over that instance variable.

In [1]:
class Reverse:
    """Iterator for looping over a sequence backwards."""
    def __init__(self, data):
        self.data = data

    def __iter__(self):
        return ReverseIterator(self)
    
class ReverseIterator:
    def __init__(self, reverseObject):
        self.index = len(reverseObject.data)
        self.r = reverseObject
        
    def next(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.r.data[self.index]

In [2]:
rev = Reverse('spam')
print rev

<__main__.Reverse instance at 0x10a8035f0>


In [3]:
iter(rev)

<__main__.ReverseIterator instance at 0x10a803680>

In [4]:
for char in rev:
    print char

m
a
p
s


- The simplest sort of container object will have its own __next__ method that, when called, returns to the for loop the next element in the container. When there are no more elements in the container, it raises a StopIteration exception (see 8.4 in [Python tutorial](https://docs.python.org/2/tutorial/) ) instead of returning an element. The for loop terminates when it gets this exception.

- In general, however, the container object doesn’t need to have its own __next__ method. Instead, it may assign the job of picking the next element to a separate object, called an __iterator__.

Note: the next( ) method has been renamed to \__next\__( ) in Python 3.

- In general, an iterator is any object that defines a suitable __next__ method. When an iterator object’s __next__ method is invoked, the method should return the next element of some collection – whatever that may mean. How the __next__ method is written defines the order in which the elements of a collection are iterated over in a for loop.

- Your collection appoints an iterator by defining an \__iter\__ method that returns an instance of an iterator object.

If the collection has its own next method, the collection’s \__iter\__ method can return self; the container will serve as its own iterator. Note that in the following example, the __Reverse__ class is both the container and the iterator object. But in general, the iterator can be a separate object from the container.

In [5]:
class Reverse:
    """Iterator for looping over a sequence backwards."""
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    def __iter__(self):
        return self

    def next(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]

In [6]:
rev = Reverse('apple')
iter(rev)
for char in rev:
    print char

e
l
p
p
a


Here's another example:

In [ ]:
class ThreeElementContainer:
    def __init__(self, a = 0, b = 0, c = 0):
        self.a = a
        self.b = b
        self.c = c
        self.i = 0
    
    def __iter__(self):
        print "iter called"
        return self
        
    def next(self):
        print "next called"
        if self.i == 0:
            el = self.a
        elif self.i == 1:
            el = self.b
        elif self.i == 2:
            el = self.c
        else:
            print "raised an exception"
            raise StopIteration
        self.i += 1
        return el
        
    def __str__(self):
        return "[" + str(self.a) + ", " + str(self.b) + ", " + str(self.c)+ "]"
        
t = ThreeElementContainer(5,10,15)
#print t
for el in t:
    print el

## Generators
The generator is the elegant brother of iterator that allows you to write iterators like the one you saw earlier, but in a much easier syntax where you do not have to write classes with \__iter\__( ) and next( ) methods.

Now let's write a __generator__ every_other(data) that yields every other
element of the data. 

In [7]:
def every_other(data):
    for index in range(0,len(data),2):
        yield data[index]
        
for char in every_other("supercalifragilisticexpialidocious"):
    print char,

s p r a i r g l s i e p a i o i u


The magic word with generators is __yield__. There is no return statement in the function every_other. The return value of the function will actually be a generator. Inside the for loop when the execution reaches the yield statement, the value of data[index] is returned and the generator state is suspended. During the second next call, the generator resumes from the index at which it stopped earlier and increases this index by one. It continues with the for loop and comes to the yield statement again.

__yield__ basically replaces the return statement of a function but rather provides a result to its caller without destroying local variables. Thus, in the next iteration, it can work on this local variable value again. So unlike a normal function that you have seen before, where on each call it starts with new set of variables - a generator will resume the execution where it was left off.

Now let's rewrite the ThreeElementContainer class using a generator:

In [ ]:
class ThreeElementContainer:
    def __init__(self, a = 0, b = 0, c = 0):
        self.a = a
        self.b = b
        self.c = c
#        self.i = 0
    
#    def __iter__(self):
#        print "iter called"
#        return self

    def __iter__(self):
        return self.generator()
    
    def generator(self):
        yield self.a
        yield self.b
        yield self.c
        
#    def next(self):
#        print "next called"
#        if self.i == 0:
#            el = self.a
#        elif self.i == 1:
#            el = self.b
#        elif self.i == 2:
#            el = self.c
#        else:
#            print "raised an exception"
#            raise StopIteration
#        self.i += 1
#        return el
        
    def __str__(self):
        return "[" + str(self.a) + ", " + str(self.b) + ", " + str(self.c)+ "]"
        
t = ThreeElementContainer(5,10,15)
#print t
for el in t:
    print el


## Generator expressions
The generator expressions are the generator equivalent of a list comprehension, but with parentheses instead of square brackets. Just like a list comprehension returns a list, a generator expression will return a generator.

In [8]:
squares = (x * x for x in range(1,10))
print(type(squares))
print(list(squares))

<type 'generator'>
[1, 4, 9, 16, 25, 36, 49, 64, 81]


We can also create the same generator every_other (as above) in a single line using a
lambda function and a generator expression.

In [1]:
every_other2 = lambda data: (char for char in data[::2])
for char in every_other2("supercalifragilisticexpialidocious"):
    print char,

print ""

# a more memory-efficient solution
every_other3 = lambda data: (data[index] for index in xrange(0,len(data),2)) # range would work, too
for char in every_other3("supercalifragilisticexpialidocious"):
    print char,

s p r a i r g l s i e p a i o i u 
s p r a i r g l s i e p a i o i u
